# Fluxo de Carga Linearizado

## Imports

In [32]:
import numpy as np
import pandas as pd

## Newton Linear Class

In [33]:
class newtonLinear():
    
    def __init__(self, bus_df, branch_df, referencia, perdas = False):
        
        self.bus_df    = bus_df
        self.branch_df = branch_df
        
        self.bus_len = len(self.bus_df)
        self.bra_len = len(self.branch_df)
        
        self.referencia = referencia
        self.perdas     = perdas
        
        
    def createMatrix(self):
        
        self.P     = np.zeros(self.bus_len, dtype=float)
        self.B     = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.B_inf = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.a     = np.zeros(self.bus_len, dtype=float)
        
        self.xm = np.zeros((self.bus_len, self.bus_len), dtype=float)
        
        if self.perdas:
            self.X  = np.zeros((self.bus_len, self.bus_len), dtype=float)
            self.R  = np.zeros((self.bus_len, self.bus_len), dtype=float)
        
        for i in range(self.bus_len):
            
            self.P[i]  = self.bus_df.loc[i+1,['Pg']].to_numpy(dtype=float)[0] - self.bus_df.loc[i+1,['Pc']].to_numpy(dtype=float)[0]
        
        for i in range(self.bra_len):
            
            k = self.branch_df.loc[i,['From']].to_numpy(dtype=int)[0]-1
            m = self.branch_df.loc[i,['To']].to_numpy(dtype=int)[0]-1

            r = self.branch_df.loc[i,['r']].to_numpy(dtype=float)[0]
            x = self.branch_df.loc[i,['x']].to_numpy(dtype=float)[0]

            self.xm[k, m] = 1/x
            self.xm[m, k] = 1/x
            
            if self.perdas:
            
                self.X[k, m] = x
                self.X[m, k] = x

                self.R[k, m] = r
                self.R[m, k] = r
            
        for i in range(self.bus_len):
            for j in range(self.bus_len):
                
                if i != j:
                    
                    self.B[i, j]     = - self.xm[i, j]
                    self.B_inf[i, j] = - self.xm[i, j]
                    
        for i in range(self.bus_len):
            
            soma = 0
            for j in range(self.bus_len):
                
                if i != j:
                    
                    soma += self.xm[i, j]
                    
            self.B[i, i]     = soma
            self.B_inf[i, i] = soma
            
        self.B_inf[self.referencia-1, self.referencia-1] = np.Inf
        
        if self.perdas:
            
            self.g = np.zeros((self.bus_len, self.bus_len), dtype=float)
            
            for i in range(self.bus_len):
                for j in range(self.bus_len):
                    
                    if i != j:
                        
                        if self.R[i, j]**2 + self.X[i, j]**2 != 0:
                        
                            self.g[i, j] = self.R[i, j]/(self.R[i, j]**2 + self.X[i, j]**2)
            
            
        
    def angleCal(self):
        
        self.b_inv = np.linalg.inv(self.B_inf)

        self.a = np.dot(self.b_inv, self.P)
        
        self.bus_df['Theta'] = self.a
        
        if self.perdas:    
            
            self.ad = np.zeros((self.bus_len, self.bus_len), dtype=float)
            
            for i in range(self.bus_len):
                for j in range(self.bus_len):
                    
                    if i != j:
            
                     self.ad[i, j] = self.a[i] - self.a[j]
        
    def powerCal(self):
        
        self.P = np.dot(self.B, self.a)
        Pg = np.zeros(self.bus_len, dtype=float)
        Pc = np.zeros(self.bus_len, dtype=float)
        
        
        if self.perdas:
            
            self.p1  = np.zeros(self.bus_len, dtype=float)
            self.pot = np.zeros(self.bus_len, dtype=float)
            self.Pp  = np.zeros((self.bus_len, self.bus_len), dtype=float)
            
            self.branch_df['Perdas'] = np.zeros(self.bra_len, dtype=float)
            
            for i in range(self.bus_len):
                
                soma = 0 
                for j in range(self.bus_len):
                    
                    if i != j:
                        
                        soma += -(self.g[i, j]*self.ad[i, j]**2)/2
                        
                self.p1[i] = soma
                
            self.pot = self.P + self.p1
            
            self.a = np.dot(self.b_inv, self.pot)
            self.P = np.dot(self.B, self.a)
            
            self.bus_df['Theta'] = self.a
            
            for i in range(self.bus_len):
                self.P[i] = self.P[i] - self.p1[i]
                
            for i in range(self.bus_len):
                for j in range(self.bus_len):
                    
                    if i != j:
                        self.Pp[i, j] = self.P[i] + self.P[j]
                        
                        
            for i in range(self.bra_len):
                
                k = self.branch_df.loc[i,['From']].to_numpy(dtype=int)[0]-1
                m = self.branch_df.loc[i,['To']].to_numpy(dtype=int)[0]-1
                
                self.branch_df.loc[i,['Perdas']] = self.Pp[k, m]
                
                
                
                        
        for i in range(self.bus_len):
            
            if self.P[i] > 0:
                
                Pg[i] = self.P[i]
                
            else:
                
                Pc[i] = np.abs(self.P[i])
                
        self.bus_df['Pg'] = Pg
        self.bus_df['Pc'] = Pc

## Programa

### Sem Perdas

In [34]:
BusData = np.loadtxt('ieee5bus.txt',skiprows=2)
BranchData = np.loadtxt('ieee5branch.txt',skiprows=2)

In [35]:
Pb = 100                # Potência base para conversão para PU
Perdas = False          # Sem perdas
Barra_referencia = 1    # Barra Referencia

In [36]:
for i in range(len(BusData)):
    BusData[i, 4:7] = BusData[i, 4:7] / Pb


In [37]:
i = [i+1 for i in range(len(BusData))]

df_bus = pd.DataFrame(BusData[:, 0],columns=['Bus'], index = i)

tipo = ["PQ", "NaN", "PV", "VO"]
tipo_b = []
for i in range(len(BusData)):
  tipo_b.append(tipo[int(BusData[i, 1])])

df_bus['Type'] = tipo_b

df_bus['Pg']    = BusData[:, 6]
df_bus['Pc']    = BusData[:, 4]
df_bus['Qg']    = BusData[:, 7]
df_bus['Qc']    = BusData[:, 5]
df_bus['V']     = BusData[:, 2]
df_bus['Theta'] = BusData[:, 3]
df_bus['Bsh']   = BusData[:, 8]

df_bus

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,0.0,0.00,-0.0,0.00,1.0,0.0,0.0
2,2.0,PQ,0.0,0.40,0.0,0.20,1.0,0.0,0.0
3,3.0,PQ,0.0,0.25,0.0,0.15,1.0,0.0,0.0
4,4.0,PQ,0.0,0.40,0.0,0.20,1.0,0.0,0.0
5,5.0,PQ,0.0,0.50,0.0,0.20,1.0,0.0,0.0


In [38]:
df_branch = pd.DataFrame(BranchData, columns = ['From', 'To', 'r', 'x', 'Bsh', 'Tap', 'Phi'])
df_branch

,From,To,r,x,Bsh,Tap,Phi
0,1.0,2.0,0.05,0.11,0.02,1.0,0.0
1,1.0,3.0,0.05,0.11,0.02,1.0,0.0
2,1.0,5.0,0.03,0.08,0.02,1.0,0.0
3,2.0,3.0,0.04,0.09,0.02,1.0,0.0
4,2.0,5.0,0.04,0.09,0.02,1.0,0.0
5,3.0,4.0,0.06,0.13,0.03,1.0,0.0
6,4.0,5.0,0.04,0.09,0.02,1.0,0.0


In [39]:
nl = newtonLinear(df_bus, df_branch, Barra_referencia, Perdas)
nl.createMatrix()
nl.angleCal()
nl.powerCal()

In [40]:
nl.bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,1.55,0.00,-0.0,0.00,1.0,0.000000,0.0
2,2.0,PQ,0.00,0.40,0.0,0.20,1.0,-0.048854,0.0
3,3.0,PQ,0.00,0.25,0.0,0.15,1.0,-0.048429,0.0
4,4.0,PQ,0.00,0.40,0.0,0.20,1.0,-0.072550,0.0
5,5.0,PQ,0.00,0.50,0.0,0.20,1.0,-0.053249,0.0


In [41]:
bus_bonito = nl.bus_df.copy()

In [42]:
precisao = 3

for i in range(len(nl.bus_df)):

    bus_bonito["Theta"][i+1] = round(np.degrees(nl.bus_df["Theta"][i+1]), precisao)

    bus_bonito["Pg"][i+1] = round(nl.bus_df["Pg"][i+1], precisao)
    bus_bonito["Pc"][i+1] = round(nl.bus_df["Pc"][i+1], precisao)
    bus_bonito["Qg"][i+1] = round(nl.bus_df["Qg"][i+1], precisao)
    bus_bonito["Qc"][i+1] = round(nl.bus_df["Qc"][i+1], precisao)

    bus_bonito["V"][i+1] = round(nl.bus_df["V"][i+1], precisao)   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A va

In [43]:
bus_bonito

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,1.55,0.00,-0.0,0.00,1.0,0.000,0.0
2,2.0,PQ,0.00,0.40,0.0,0.20,1.0,-2.799,0.0
3,3.0,PQ,0.00,0.25,0.0,0.15,1.0,-2.775,0.0
4,4.0,PQ,0.00,0.40,0.0,0.20,1.0,-4.157,0.0
5,5.0,PQ,0.00,0.50,0.0,0.20,1.0,-3.051,0.0


### Com Perdas



In [44]:
BusData = np.loadtxt('ieee5bus.txt',skiprows=2)
BranchData = np.loadtxt('ieee5branch.txt',skiprows=2)

In [45]:
Pb = 100                # Potência base para conversão para PU
Perdas = True           # Com perdas
Barra_referencia = 1    # Barra Referencia

In [46]:
for i in range(len(BusData)):
    BusData[i, 4:7] = BusData[i, 4:7] / Pb

In [47]:
i = [i+1 for i in range(len(BusData))]

df_bus = pd.DataFrame(BusData[:, 0],columns=['Bus'], index = i)

tipo = ["PQ", "NaN", "PV", "VO"]
tipo_b = []
for i in range(len(BusData)):
  tipo_b.append(tipo[int(BusData[i, 1])])

df_bus['Type'] = tipo_b

df_bus['Pg']    = BusData[:, 6]
df_bus['Pc']    = BusData[:, 4]
df_bus['Qg']    = BusData[:, 7]
df_bus['Qc']    = BusData[:, 5]
df_bus['V']     = BusData[:, 2]
df_bus['Theta'] = BusData[:, 3]
df_bus['Bsh']   = BusData[:, 8]

df_bus

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,0.0,0.00,-0.0,0.00,1.0,0.0,0.0
2,2.0,PQ,0.0,0.40,0.0,0.20,1.0,0.0,0.0
3,3.0,PQ,0.0,0.25,0.0,0.15,1.0,0.0,0.0
4,4.0,PQ,0.0,0.40,0.0,0.20,1.0,0.0,0.0
5,5.0,PQ,0.0,0.50,0.0,0.20,1.0,0.0,0.0


In [48]:
df_branch = pd.DataFrame(BranchData, columns = ['From', 'To', 'r', 'x', 'Bsh', 'Tap', 'Phi'])
df_branch

,From,To,r,x,Bsh,Tap,Phi
0,1.0,2.0,0.05,0.11,0.02,1.0,0.0
1,1.0,3.0,0.05,0.11,0.02,1.0,0.0
2,1.0,5.0,0.03,0.08,0.02,1.0,0.0
3,2.0,3.0,0.04,0.09,0.02,1.0,0.0
4,2.0,5.0,0.04,0.09,0.02,1.0,0.0
5,3.0,4.0,0.06,0.13,0.03,1.0,0.0
6,4.0,5.0,0.04,0.09,0.02,1.0,0.0


In [49]:
nl = newtonLinear(df_bus, df_branch, Barra_referencia, Perdas)
nl.createMatrix()
nl.angleCal()
nl.powerCal()

In [50]:
nl.bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,1.581178,0.00,-0.0,0.00,1.0,0.000000,0.0
2,2.0,PQ,0.000000,0.40,0.0,0.20,1.0,-0.049391,0.0
3,3.0,PQ,0.000000,0.25,0.0,0.15,1.0,-0.048999,0.0
4,4.0,PQ,0.000000,0.40,0.0,0.20,1.0,-0.073209,0.0
5,5.0,PQ,0.000000,0.50,0.0,0.20,1.0,-0.053823,0.0


In [51]:
bus_bonito = nl.bus_df.copy()

In [52]:
precisao = 3

for i in range(len(nl.bus_df)):

    bus_bonito["Theta"][i+1] = round(np.degrees(nl.bus_df["Theta"][i+1]), precisao)

    bus_bonito["Pg"][i+1] = round(nl.bus_df["Pg"][i+1], precisao)
    bus_bonito["Pc"][i+1] = round(nl.bus_df["Pc"][i+1], precisao)
    bus_bonito["Qg"][i+1] = round(nl.bus_df["Qg"][i+1], precisao)
    bus_bonito["Qc"][i+1] = round(nl.bus_df["Qc"][i+1], precisao)

    bus_bonito["V"][i+1] = round(nl.bus_df["V"][i+1], precisao)   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A va

In [53]:
bus_bonito

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,1.581,0.00,-0.0,0.00,1.0,0.000,0.0
2,2.0,PQ,0.000,0.40,0.0,0.20,1.0,-2.830,0.0
3,3.0,PQ,0.000,0.25,0.0,0.15,1.0,-2.807,0.0
4,4.0,PQ,0.000,0.40,0.0,0.20,1.0,-4.195,0.0
5,5.0,PQ,0.000,0.50,0.0,0.20,1.0,-3.084,0.0
